In [1]:
import nltk
import random
import os
import json
from nltk.stem import  PorterStemmer
import random
import pickle

In [2]:
ps=PorterStemmer()

In [3]:
def load_dataset():  
    dataset=[]
    for root, dirs, files in os.walk('../Dataset'):
        for directory in dirs:
            for root_f, dirs_f, files_f in os.walk(os.path.join(root,directory)):
                for each_file in files_f:
                    with open(os.path.join(root,directory)+'/'+each_file) as f:
                        data_temp=json.load(f)    
                    if(each_file == 'pos' or each_file == 'sli_pos'):
                        dataset+=[(sentence,'pos') for sentence in data_temp]
                    elif(each_file == 'neg' or each_file == 'sli_neg'):
                        dataset+=[(sentence,'neg') for sentence in data_temp]
                    elif(each_file == 'neutral'):
                        dataset+=[(sentence,'neutral') for sentence in data_temp]
    return dataset

In [4]:
def to_unigram(data):
    dataset=[]
    dataset+=[([items for items in sentence.split()],polarity) for sentence,polarity in data]    
    return dataset    

In [5]:
#execution
labeled_data=load_dataset()    #loading labeled sentences
random.shuffle(labeled_data)
labeled_data=to_unigram(labeled_data)    #converting to unigram

In [6]:
#loading vocab
with open('vocab') as f:
    vocab=json.load(f)

In [7]:
print labeled_data[1744]

([u'never', u'gave', u'chance', u'superiors', u'question', u'work'], 'neutral')


In [8]:
print len(labeled_data)

179644


In [9]:
print 'Length Of Vocab : ' + str(len(vocab))

Length Of Vocab : 307


In [10]:
vocab[:10]

[u'ridicul',
 u'lack',
 u'poorli',
 u'bright',
 u'unorgan',
 u'abil',
 u'rough',
 u'hate',
 u'enthusiast',
 u'slow']

In [11]:
def extract_features(document):       #features are bag of words. document is a list of words of a sentence 
    features = {}
    document= [ps.stem(word) for word in document]
    for word in vocab:
        features['contains(%s)' % word] = (word in document)
    return features        

In [12]:
train_test = nltk.classify.apply_features(extract_features, labeled_data)

In [13]:
#dividing into training and test data
train_set=train_test[:-10000]
test_set=train_test[-10000:]

In [14]:
print labeled_data[10] 
train_set[10]

([u'mind', u'teaching', u'anything', u'want', u'learn', u'fields', u'offer'], 'neutral')


({u'contains(abil)': False,
  u'contains(abl)': False,
  u'contains(activ)': False,
  u'contains(affect)': False,
  u'contains(afraid)': False,
  u'contains(allow)': False,
  u'contains(amaz)': False,
  u'contains(ambiti)': False,
  u'contains(angri)': False,
  u'contains(angrili)': False,
  u'contains(apprais)': False,
  u'contains(appreci)': False,
  u'contains(attract)': False,
  u'contains(avail)': False,
  u'contains(award)': False,
  u'contains(awesom)': False,
  u'contains(bad)': False,
  u'contains(badli)': False,
  u'contains(balanc)': False,
  u'contains(benefit)': False,
  u'contains(best)': False,
  u'contains(better)': False,
  u'contains(bore)': False,
  u'contains(break)': False,
  u'contains(bright)': False,
  u'contains(brilliant)': False,
  u'contains(broken)': False,
  u'contains(brutal)': False,
  u'contains(bunch)': False,
  u'contains(busi)': False,
  u'contains(cannot)': False,
  u'contains(cant)': False,
  u'contains(challeng)': False,
  u'contains(comfort)': Fa

In [ ]:
classifier = nltk.NaiveBayesClassifier.train(train_set)
with open('senti_naive_bayes_unigram_model','w') as f:
    pickle.dump(classifier,f)

In [ ]:
#with open('naive_bayes_unigram_model') as f:
#    classifier=pickle.load(f)
accuracy=nltk.classify.accuracy(classifier, test_set)
with open('senti_accuracy_unigram_model','w') as f:
    json.dump(accuracy,f)

In [38]:
print 'Accuracy : '+ str(accuracy)

Accuracy : 0.8336


In [39]:
classifier.show_most_informative_features(25)

Most Informative Features
         contains(great) = True              pos : neg    =   1634.1 : 1.0
       contains(terribl) = True              neg : pos    =    432.6 : 1.0
       contains(horribl) = True              neg : pos    =    279.7 : 1.0
         contains(worst) = True              neg : pos    =    278.1 : 1.0
        contains(unfair) = True              neg : pos    =    223.1 : 1.0
       contains(exhaust) = True              neg : pos    =    164.1 : 1.0
          contains(best) = True              pos : neutra =    159.2 : 1.0
          contains(fail) = True              neg : pos    =    158.2 : 1.0
     contains(unrealist) = True              neg : pos    =    149.4 : 1.0
          contains(amaz) = True              pos : neutra =    145.4 : 1.0
          contains(good) = True              pos : neg    =    145.3 : 1.0
         contains(miser) = True              neg : pos    =    134.6 : 1.0
        contains(imposs) = True              neg : pos    =    125.5 : 1.0

In [41]:
classifier.classify(extract_features('very uncomfortable in working'.split()))

'neg'